# Fetch and save prerequisites for all courses in the dataset

Idea and structure by me and most code by Chatgpt

In [1]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
import re
import json

## Download the prerequisites data from the course page  

In [ ]:
changed_rows = 0
dropped_rows = 0


# Function to process 'KURSKODE'
def transform_kurskode(code):
    global changed_rows
    # Check if it starts with a number
    if not code[0].isdigit():
        return np.nan  # Mark for dropping
    original_code = code
    # Pad with 0 if less than 5 characters
    if len(code) < 5:
        code = code.zfill(5)
    # Trim to 5 characters if more than 5
    elif len(code) > 5:
        code = code[:5]
    if code != original_code:
        changed_rows += 1
    return code


def process_kurskode(code):
    # Load your CSV file into a DataFrame
    df = pd.read_csv('20221012_karakterdata til studenterprojekt.csv')

    # Initialize counters for statistics
    total_rows = len(df)


    # Apply the transformation
    df['KURSKODE'] = df['KURSKODE'].astype(str).apply(transform_kurskode)

    # Drop rows where KURSKODE is NaN (i.e., where it started with a non-digit)
    df.dropna(subset=['KURSKODE'], inplace=True)

    # Drop duplicate rows based on 'KURSKODE' to retain unique values
    df.drop_duplicates(subset=['KURSKODE'], inplace=True)

    # Update the dropped_rows count after dropping duplicates
    dropped_rows = total_rows - len(df)

    # Convert the 'KURSKODE' column to a numpy array
    kurskode_array = df['KURSKODE'].to_numpy()

    # Print statistics
    print(f"Total rows read: {total_rows}")
    print(f"Rows changed: {changed_rows}")
    print(f"Rows dropped: {dropped_rows}")
    print(f"Unique Rows remaining: {len(df)}")

    # Save the numpy array if needed
    np.save('kurskode_array.npy', kurskode_array)


# Function to extract academic prerequisites from a course page
def extract_prerequisites(course_code, start_year=2023, end_year=2005):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
        'Accept-Language': 'da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Connection': 'keep-alive',
        'Cookie': 'ASP.NET_SessionId=u3wwy3s2suujmsoep02sby0l; SRV_ID=2; {DTUCoursesPublicLanguage}=en-GB',
    }

    cookies = {
        'ASP.NET_SessionId': 'u3wwy3s2suujmsoep02sby0l',
        'SRV_ID': '2',
        '{DTUCoursesPublicLanguage}': 'en-GB',
    }

    # First, check the latest version of the course (current year)
    url = f'https://kurser.dtu.dk/course/{course_code}'
    response = requests.get(url, headers=headers, cookies=cookies)

    if response.status_code != 200:
        return None, f"Failed to retrieve course {course_code}"

    soup = BeautifulSoup(response.text, 'html.parser')

    # Check for multiple possible labels
    possible_labels = ["Academic prerequisites", "Recommended prerequisites", "Mandatory Prerequisites"]

    for label_text in possible_labels:
        prerequisites_label = soup.find('label', text=label_text)
        if prerequisites_label:
            prerequisites_cell = prerequisites_label.find_next('td')
            if prerequisites_cell:
                prerequisites_text = prerequisites_cell.get_text(strip=True)
                return prerequisites_text, None

    # If no prerequisites found in the latest year, try going back to previous years
    for year in range(start_year, end_year - 1, -1):
        historical_url = f'https://kurser.dtu.dk/course/{year}-{year+1}/{course_code}'
        print(f"Checking for course {course_code} in academic year {year}-{year+1}")
        response = requests.get(historical_url, headers=headers, cookies=cookies)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            for label_text in possible_labels:
                prerequisites_label = soup.find('label', text=label_text)
                if prerequisites_label:
                    prerequisites_cell = prerequisites_label.find_next('td')
                    if prerequisites_cell:
                        prerequisites_text = prerequisites_cell.get_text(strip=True)
                        return prerequisites_text, None

    # If no prerequisites found across years
    return None, f"Prerequisites not found for course {course_code} even in older versions"


# List of course codes to process
course_codes = np.load('kurskode_array.npy', allow_pickle=True)
# course_codes = course_codes[:100]  # Limit to 10 for testing

# Arrays to store course data
general_rule_courses = []
special_treatment_courses = []  # This will store both the course code and the reason

# Process each course
for course_code in course_codes:
    prerequisites_text, special_case = extract_prerequisites(course_code)

    if prerequisites_text:
        # Append the course and its prerequisites text to the general rule courses
        general_rule_courses.append((course_code, prerequisites_text))
    else:
        # Append both course code and reason to special_treatment_courses
        special_treatment_courses.append({
            "course_code": course_code,
            "reason": special_case
        })

# Save the general rule courses to a numpy array and CSV
general_rule_array = np.array(general_rule_courses, dtype=object)
np.save('general_rule_array.npy', general_rule_array)
df = pd.DataFrame(general_rule_array, columns=['Course', 'Prerequisites'])
df.to_csv('general_rule_courses.csv', index=False)

# Save the general rule courses in JSON Lines format
with open('general_rule_courses.jsonl', 'w', encoding='utf-8') as f:
    for course in general_rule_courses:
        f.write(json.dumps(course, ensure_ascii=False) + "\n")

# Print stats
print(f"Total courses processed: {len(course_codes)}")
print(f"Courses with academic prerequisites text saved: {len(general_rule_courses)}")
print(f"Courses for special treatment: {len(special_treatment_courses)}")

# Save the special treatment courses to a numpy array
special_treatment_array = np.array(special_treatment_courses, dtype=object)
np.save('special_treatment_array.npy', special_treatment_array)

# Convert to DataFrame and save as CSV
df = pd.DataFrame(special_treatment_courses)
df.to_csv('special_treatment_courses.csv', index=False)

# Save the special treatment courses in JSON Lines format
with open('special_treatment_courses.jsonl', 'w', encoding='utf-8') as f:
    for course in special_treatment_courses:
        f.write(json.dumps(course, ensure_ascii=False) + "\n")


print("\nCourses requiring special treatment:")
for course in special_treatment_courses:
    print(f"Course {course['course_code']}: {course['reason']}")


## Extract prerequisites from the following rules                                                     
Commas(,) and periods(.) mean AND.                                                                  
Slashes( /) and the word “eller” mean OR.                                                           
Course numbers prefixed by a dash(-) should have the dash removed.          
Course numbers should be exactly 5 digits, with a leading 0 added if they are only 4 digits.                                                 

In [2]:
# Load the CSV file
file_path = 'general_rule_courses.csv'
df = pd.read_csv(file_path)

# Load the special treatment courses
special_treatment_file_path = 'special_treatment_courses.csv'
special_treatment_df = pd.read_csv(special_treatment_file_path)


# Function to clean course numbers by removing "-" and ensuring 5 digits with leading zeros
def clean_course_number(course):
    course = course.strip()  # remove any surrounding whitespace
    course = course.lstrip('-')  # remove leading dash if present
    if len(course) == 4:
        course = '0' + course  # add leading zero if only 4 digits
    return course


# Function to process prerequisites with parentheses for grouping
def parse_prerequisites(prerequisites_text):
    # Replace "eller" with "/"
    prerequisites_text = prerequisites_text.replace('eller', '/')

    # Split based on AND operators (commas and periods)
    parts = re.split(r'[,\.]', prerequisites_text)
    parsed_prerequisites = []
    invalid_cases = []

    for part in parts:
        part = part.strip()  # Clean up any extra spaces
        # Split on OR indicators within each part
        sub_parts = re.split(r'[\/]', part)
        cleaned_sub_parts = []

        for sub_part in sub_parts:
            # Extract valid course numbers (should be 4 or 5 digits)
            course_numbers = re.findall(r'\d{4,5}', sub_part)
            if course_numbers:
                cleaned_sub_parts.append(' or '.join([clean_course_number(num) for num in course_numbers]))
            else:
                invalid_cases.append(sub_part.strip())  # Capture invalid cases

        if cleaned_sub_parts:
            # Join sub-parts (OR parts) with parentheses
            parsed_prerequisites.append(f"({' or '.join(cleaned_sub_parts)})")

    # Join the AND parts
    return ' and '.join(parsed_prerequisites), invalid_cases


# Prepare lists to hold different categories
valid_prerequisites = []
no_prerequisites = []
further_processing = []

# Process each course and its prerequisites
for index, row in df.iterrows():
    course_code = str(row['Course'])
    prerequisites_text = row['Prerequisites']

    # Check if the prerequisites are "None" or empty
    if pd.isna(prerequisites_text) or prerequisites_text.strip().lower() == 'none':
        no_prerequisites.append({"course_code": course_code})
    else:
        parsed, invalid = parse_prerequisites(prerequisites_text)

        # If there are valid parsed prerequisites, add them to the valid list
        if parsed and not invalid:
            valid_prerequisites.append({"course_code": course_code, "parsed_prerequisites": parsed})
        elif parsed:
            # If valid prerequisites are found, only add to the valid list, not further processing
            valid_prerequisites.append({"course_code": course_code, "parsed_prerequisites": parsed})
        else:
            # If no valid prerequisites, add to further processing
            further_processing.append({"course_code": course_code, "invalid_prerequisites": invalid})

# Add the courses from the special_treatment_courses.csv to the no_prerequisites list
for index, row in special_treatment_df.iterrows():
    course_code = str(row['course_code'])
    no_prerequisites.append({"course_code": course_code})

# Save files for no prerequisites, valid prerequisites, and further processing

# Save no prerequisites to JSON Lines
no_prerequisites_file = 'no_prerequisites.jsonl'
with open(no_prerequisites_file, 'w', encoding='utf-8') as f:
    for entry in no_prerequisites:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

# Save valid prerequisites to JSON Lines
valid_prerequisites_file = 'valid_prerequisites.jsonl'
with open(valid_prerequisites_file, 'w', encoding='utf-8') as f:
    for entry in valid_prerequisites:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

# Save courses needing further processing to JSON Lines
further_processing_file = 'further_processing.jsonl'
with open(further_processing_file, 'w', encoding='utf-8') as f:
    for entry in further_processing:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"Total courses processed: {len(df)+len(special_treatment_df)}")
print(f"Valid prerequisites: {len(valid_prerequisites)}")
print(f"No prerequisites: {len(no_prerequisites)}")
print(f"Needing further processing: {len(further_processing)}")


Total courses processed: 1945
Valid prerequisites: 1280
No prerequisites: 383
Needing further processing: 282


## Save the prerequisites in JSON Lines format

In [6]:

# Load the CSV file for validation (df) and the valid prerequisites JSON Lines file
file_path = 'general_rule_courses.csv'
df = pd.read_csv(file_path)

valid_prerequisites_file = 'valid_prerequisites.jsonl'

# Load valid prerequisites from JSONL
valid_prerequisites = []
with open(valid_prerequisites_file, 'r', encoding='utf-8') as f:
    for line in f:
        valid_prerequisites.append(json.loads(line))

# Convert valid_prerequisites list into a DataFrame
valid_prerequisites_df = pd.DataFrame(valid_prerequisites)

# Convert both 'Course' and 'course_code' to strings for a consistent merge
df['Course'] = df['Course'].astype(str)
valid_prerequisites_df['course_code'] = valid_prerequisites_df['course_code'].astype(str)

# Merge df with valid prerequisites DataFrame
merged_df = pd.merge(df, valid_prerequisites_df, how='left', left_on='Course', right_on='course_code')

# Replace NaN in the 'parsed_prerequisites' column with empty strings for courses with no prerequisites
merged_df['parsed_prerequisites'].fillna('', inplace=True)

# Drop the duplicate 'course_code' column
merged_df.drop(columns=['course_code'], inplace=True)
merged_df.to_csv('validation.csv')

# Create a dictionary for easy lookup
course_prerequisites = {}
for index, row in merged_df.iterrows():
    course_code = str(row['Course'])
    prerequisites = row['parsed_prerequisites'] if row['parsed_prerequisites'] else ''
    course_prerequisites[course_code] = prerequisites

# You can save the final data structure (course_prerequisites) to a JSON file for persistence if needed
course_prerequisites_file = 'course_prerequisites.json'
with open(course_prerequisites_file, 'w', encoding='utf-8') as f:
    json.dump(course_prerequisites, f, ensure_ascii=False, indent=4)

/var/folders/p3/1syz8hg56wvffghj9y_06hz80000gn/T/ipykernel_52458/2599797649.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['parsed_prerequisites'].fillna('', inplace=True)


## Validate the prerequisites for a few courses

In [4]:
merged_df.head(10)

,Course,Prerequisites,parsed_prerequisites
0,2402,"01001/01003/01002/01004/01005/01901, At the la...",(01001 or 01003 or 01002 or 01004 or 01005 or ...
1,28213,26400.26222,(26400) and (26222)
2,41632,"41612, Product Design and\r\nDocumentation",(41612)
3,41656,41661/41650/41659/41680.­41684,(41661 or 41650 or 41659 or 41680) and (41684)
4,41661,41650/41657/41659/41681/41683/41680/41686.­416...,(41650 or 41657 or 41659 or 41681 or 41683 or ...
5,41667,41650/41657/41659/41681/41683/41658/41680/4168...,(41650 or 41657 or 41659 or 41681 or 41683 or ...
6,41737,"Knowledge of strength of materials, and plasti...",(41501) and (41650)
7,41738,Knowlegde of polymer materials and polymer pro...,(41736 or 41737 or 41785 or 42231 or 42233 or ...
8,41744,"41703/41781.41650/41659, or equivalent Product...",(41703 or 41781) and (41650 or 41659)
9,41747,Knowledge of basic process technology 42201/42...,(42201 or 42911 or 41704 or 41784) and (41659)...


## Test the prerequisites

In [5]:
# Load the JSON file into a Python dictionary
with open(course_prerequisites_file, 'r', encoding='utf-8') as f:
    course_prerequisites = json.load(f)

def check_prerequisites(course_code, passed_courses, course_prerequisites):
    # Get the prerequisites expression for the course
    prerequisites = course_prerequisites.get(course_code, '')

    # If there are no prerequisites, return True
    if not prerequisites:
        return True

    # For each course number, replace it with the corresponding condition (True/False)
    course_numbers = re.findall(r'\d{4,5}', prerequisites)  # Find all 4- or 5-digit numbers
    for course in course_numbers:
        prerequisites = prerequisites.replace(course, f'({course in passed_courses})')

    # Evaluate the expression
    try:
        return eval(prerequisites)
    except Exception as e:
        print(f"Error evaluating prerequisites for {course_code}: {e}")
        return False

course_code = '41525'
passed_courses = {'02002', '41564'}
print(check_prerequisites(course_code, passed_courses, course_prerequisites)) # Should be true as the student has met the prerequisites

course_code = '10240'
passed_courses = {'02002', '41564'}
print(check_prerequisites(course_code, passed_courses, course_prerequisites)) # Should be false as the student has not met  prerequisites

course_code = '12345'
passed_courses = {'02002', '41564'}
print(check_prerequisites(course_code, passed_courses, course_prerequisites)) # Should be true as the course is not covered in course_prerequisites

True
False
True
